In [2]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import ta

/Users/jayeshbhojwani/Desktop/quant dev project/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [10]:
df = pd.read_excel('eurusd-forex-data.xlsx', parse_dates=['Timestamp'])

In [11]:
df.set_index('Timestamp', inplace=True)


In [5]:
df.columns = [c.lower() for c in df.columns]


In [12]:
print(df.head())


              Open    High     Low   Close  Volume
Timestamp                                         
2005-01-04  1.3465  1.3497  1.3248  1.3270   10002
2005-01-05  1.3272  1.3304  1.3213  1.3272    9867
2005-01-06  1.3275  1.3283  1.3154  1.3181    9845
2005-01-07  1.3179  1.3252  1.3024  1.3072    9865
2005-01-11  1.3105  1.3172  1.3100  1.3118    9289


In [13]:
class RSIMeanReversion(Strategy):
    rsi_period = 14
    oversold = 30
    overbought = 70
    exit_level = 50

    def init(self):
        # Compute RSI indicator
        price = self.data.Close
        self.rsi = self.I(ta.momentum.rsi, price, self.rsi_period)

    def next(self):
        if not self.position:
            if self.rsi[-1] < self.oversold:
                self.buy()
            elif self.rsi[-1] > self.overbought:
                self.sell()
        else:
            # Exit conditions
            if self.position.is_long and self.rsi[-1] > self.exit_level:
                self.position.close()
            elif self.position.is_short and self.rsi[-1] < self.exit_level:
                self.position.close()


In [14]:
df.dropna()

,Open,High,Low,Close,Volume
Timestamp,,,,,
2005-01-04,1.34650,1.34970,1.32480,1.32700,10002
2005-01-05,1.32720,1.33040,1.32130,1.32720,9867
2005-01-06,1.32750,1.32830,1.31540,1.31810,9845
2005-01-07,1.31790,1.32520,1.30240,1.30720,9865
2005-01-11,1.31050,1.31720,1.31000,1.31180,9289
...,...,...,...,...,...
2024-06-06,1.08677,1.09020,1.08622,1.08870,33441
2024-06-07,1.08857,1.09019,1.07992,1.07994,39246
2024-06-11,1.07643,1.07735,1.07196,1.07396,34599


In [15]:
bt = Backtest(df, RSIMeanReversion, cash=10000, commission=0)


In [17]:
stats = bt.run()
print(stats)

RuntimeError: Indicator "rsi(C,14)" error. See traceback above.